In [13]:
import numpy as np
import pandas as pd 
from q_sim import *
from utils import *

In [14]:
n_trials = 1_000
n_trials_per_block = 100
n_agent = 2_000 

t_parameters = np.zeros(shape=(n_agent,2))
all_data = [] 

for agent in range(2000):
    
    reward_probs = create_reward_probs(2,n_trials_per_block,n_trials,np.array([0.1,0.5,0.9]))
    param = configuration_parameters()
    param['alpha'] = 0.3 # Maya: added to create stationary alpha
    
    sim = q_sim(
                agent,
                param,
                n_trials,
                reward_probs,
                probability_to_switch_parameters=0.005,
                max_change=np.random.choice([2,3,4])
        )
    
    df = pd.DataFrame(sim)
    all_data.append(df)
    t_parameters[agent] = list(param.values())
    
df_all_agent = pd.concat(all_data)    
df_all_agent = df_all_agent.reset_index()
df_all_agent = df_all_agent.drop(columns=['index'])  

In [15]:
# create a list of our conditions
x = 1/5
conditions = [
                (df_all_agent['alpha'] <= x), # 0
                (df_all_agent['alpha'] > 1*x) & (df_all_agent['alpha'] <= 2*x), # 1
                (df_all_agent['alpha'] > 2*x) & (df_all_agent['alpha'] <= 3*x), # 2
                (df_all_agent['alpha'] > 3*x) & (df_all_agent['alpha'] <= 4*x), # 3
                (df_all_agent['alpha'] > 4*x) # 4
    ]

# create a list of the values we want to assign for each condition
values = [0,1,2,3,4] 

# create a new column and use np.select to assign values to it using our lists as arguments
df_all_agent['alpha_bin'] = np.select(conditions, values)

# create a list of our conditions
x = 10/5
conditions = [
                (df_all_agent['beta'] <= x), # 0
                (df_all_agent['beta'] > 1*x) & (df_all_agent['beta'] <= 2*x), # 1
                (df_all_agent['beta'] > 2*x) & (df_all_agent['beta'] <= 3*x), # 2
                (df_all_agent['beta'] > 3*x) & (df_all_agent['beta'] <= 4*x), # 3
                (df_all_agent['beta'] > 4*x) # 4
            ]

# create a list of the values we want to assign for each condition
values = [0,1,2,3,4]

# create a new column and use np.select to assign values to it using our lists as arguments
df_all_agent['beta_bin'] = np.select(conditions, values)

In [16]:
df_all_agent[['agent','block','trial',
              'action','reward','alpha','beta',
              'alpha_bin','beta_bin']].to_csv('../data/artificial_trainset_2000_alphastat_03.csv',index=False)